In [1]:
import pandas as pd

In [2]:
filepath = '/Volumes/backup_128G/z_repository/TBIO_data/RequestsFromTana/20190524'

familyrelation = 'familyRelations_20190516.tsv'
familymembers = 'Familymembers_20190521.xlsx'

read_familyrelation = '{0}/{1}'.format(filepath, familyrelation)
read_familymembers = '{0}/{1}'.format(filepath, familymembers)
write_file_to = '{0}/{1}'.format(filepath, 'familyToFamily_20190525_v4.xlsx')

# Read person-family map table

In [3]:
fmDf = pd.read_excel(read_familymembers)
fmDf.shape, fmDf.head()

((4546, 2),   personStr familyStr
 0       何佩娟      何傳家系
 1       何佩珊      何傳家系
 2       何佩珍      何傳家系
 3        何傳      何傳家系
 4       何培庭      何傳家系)

In [4]:
# convertFamily = ['顏雲年家系', '顏雲年三子家系', '顏國年家系']

fmDic = {}
for idx in range(0, len(fmDf)):
    row = fmDf.loc[idx]

    person = str(row['personStr'])
    family = str(row['familyStr'])
    
#     if family in convertFamily:
#         family = '基隆顏家家族'
    
    if person not in fmDic:
        fmDic[person] = family
    elif family != fmDic[person]:
        print("Dup:", person, family, fmDic[person])

fmDic['何佩娟']

'何傳家系'

# Family relations

In [5]:
fmrelDf = pd.read_csv(read_familyrelation, delimiter='\t')
fmrelDf = fmrelDf.fillna('')
fmrelDf.shape, fmrelDf.head()

((34141, 3),   ?personVal ?relationVal    ?toPVal
 0        丁瑞彬   isFatherOf    丁瑞彬的女子1
 1        李文後   isFatherOf    李文後的男子3
 2        李文後   isFatherOf    李文後的男子2
 3        林劍清   isFatherOf        林希頂
 4        林玉雕   isFatherOf  林玉雕的男、女子8)

In [6]:
fmrelList = []
for idx in range(0, len(fmrelDf)):
    row = fmrelDf.loc[idx]

    sPerson = str(row['?personVal'])
    relation = str(row['?relationVal'])
    tPerson = str(row['?toPVal'])
    
    if sPerson == '' or tPerson == '':
        continue
        
    toRow = [sPerson, relation, tPerson]

    if toRow not in fmrelList:
        fmrelList.append(toRow)
    else:
        print("DUP!", toRow)
    
print(len(fmrelList))

34129


# Filter out inverse relations

In [7]:
def findInverse(INsPerson, INtPerson):
    for work in fmrelList:
        sPerson = work[0]
        relation = work[1]
        tPerson = work[2]
        
#         if INsPerson == sPerson and INtPerson == tPerson:
# #             print(INsPerson, INtPerson, work)
#             return relation
        if INsPerson == tPerson and INtPerson == sPerson:
#             print(INsPerson, INtPerson, work)
            return relation
    return ''

relationList = []
for work in fmrelList:
    sPerson = work[0]
    relation = work[1]
    tPerson = work[2]

    if 'has' in relation:
        relationList.append(work)
        continue
    
    OUTrelation = findInverse(sPerson, tPerson)
    if OUTrelation == '':
        relationList.append(work)
        
#     if 'has' in OUTrelation:
#         resList.append([sPerson, OUTrelation, tPerson])
#     elif OUTrelation != '':
#         resList.append(work)

print(len(relationList), relationList[0:5])

31684 [['辜顯榮', 'isFatherOf', '原配__辜顯榮長女'], ['吳運豐', 'hasFather', '吳鴻森'], ['林自炎', 'hasFather', '林歡邦'], ['吳芳英', 'hasFather', '吳鴻麟'], ['許碧瑜', 'hasFather', '許丙']]


# To find 何義家系––陳星恆家族––新竹鄭家家系––林獻堂家系

In [8]:
# 台灣五大族
oldFamilies = [
'林文察家系',
'林文明家系',
'林獻堂家系',
'林維德家系',
'林維讓家系',
'林維源家系',
'顏雲年家系', '顏雲年三子家系', '顏國年家系',
'辜顯榮家系',
'辜振甫家系',
'陳啓清家系',
'陳啓川家系'
]

In [9]:
# 新五家
newFamilies = [
'國泰蔡家家系',
'吳三連家族',
'蔡萬才家系',
'何義家系',
'何傳家系',
'何永家系',
'吳火獅家系',
'侯雨利家系',
'吳修齊家系'
]

In [10]:
def getFamily(person):
    if person not in fmDic:
        return None

    return fmDic[person]
    
def isInOldFamilies(person):
    family = getFamily(person)
    if family != None and family in oldFamilies:
        return person
    return None

def isInNewFamilies(person):
    family = getFamily(person)
    if family != None and family in newFamilies:
        return person
    return None

# Remove the pairs which one person is not in the families

In [11]:
tripleList = []
for triple in relationList:
    sPerson = triple[0]
#     relation = triple[1]
    tPerson = triple[2]
    
    sFamily = getFamily(sPerson)
    tFamily = getFamily(tPerson)
    if sFamily == tFamily:
        continue

    if sFamily != None and tFamily != None:
        familyrel = [sFamily, tFamily]
        if familyrel not in tripleList:
            tripleList.append(familyrel)

print(len(tripleList), tripleList[0:5])

2515 [['吳伯雄家系', '桃園吳家家系'], ['林維源家系', '林維讓家系'], ['徐清山家族', '陳守山家系'], ['李原宏家族', '李正明家族'], ['顏國年家系', '顏雲年家系']]


In [12]:
def findAllRelated(INsFamily):
    allRelated = []
    for triple in tripleList:
        sFamily = triple[0]
        tFamily = triple[1]
        
        if sFamily == INsFamily:
            allRelated.append(tFamily)
            
        if tFamily == INsFamily:
            allRelated.append(sFamily)
            
    return allRelated

# print(findAllRelated('林文察家系'))
# print(findAllRelated('林文明家系'))

def findFamilyConnect(connect):
    familyConnect = []
    lastFamily = connect[-1]
#     print("findFamilyConnect", connect,lastFamily)

    allRelated = findAllRelated(lastFamily)
    if len(allRelated) == 0:
        return []

    for nextFamily in allRelated:
        if nextFamily not in connect:
            newConnect = connect.copy()
            newConnect.append(nextFamily)
            familyConnect.append(newConnect)
            
#     print("findFamilyConnect Result:",familyConnect)
    return familyConnect
            

def findTwoFamilyShortestConnect(connect, tFamily):
    connects = findFamilyConnect(connect)

    found = False
    for connect in connects:
        if connect[-1] == tFamily:
#             print("found!", connects)
            found = True
            break
            
    return found, connects

def startFindTwoFamilyShortestConnect(sFamily, tFamily):
    resConnectList = []
    found, connects = findTwoFamilyShortestConnect([sFamily], tFamily)
    if found:
        for connect in connects:
            lastFamily = connect[-1]
            if lastFamily == tFamily and connect not in resConnectList:
                resConnectList.append(connect)
        return resConnectList
    
    loopConnects = connects.copy()
    for run in range(0, 6):
        print(run, len(loopConnects))
        found = False
        newLoopConnects = []
        for connect in loopConnects:
            found, connects = findTwoFamilyShortestConnect(connect, tFamily)
#             print(run, found, connect, connects)
            if len(connects) > 0:
                newLoopConnects.extend(connects)

            if found:
#                 print("found", connects)
                for newConnect in connects:
                    lastFamily = newConnect[-1]
                    if lastFamily == tFamily and newConnect not in resConnectList:
                        print("found:", newConnect)
                        resConnectList.append(newConnect)
                break
                
        if found:
            break

#         print("newLoopConnects:", newLoopConnects)
        loopConnects = newLoopConnects.copy()

    return resConnectList
    
resList = []
for newFamily in newFamilies:
    for oldFamily in oldFamilies:
        print("Checking...", newFamily, oldFamily)
        res = startFindTwoFamilyShortestConnect(newFamily, oldFamily)
        if len(res) > 0:
            resList.append(res[0])
        print(res)

# startFindTwoFamilyShortestConnect('何義家系', '林獻堂家系')
# startFindTwoFamilyShortestConnect('新竹鄭家家系', '林獻堂家系')

Checking... 國泰蔡家家系 林文察家系
0 38
found: ['國泰蔡家家系', '林文明家系', '林文察家系']
[['國泰蔡家家系', '林文明家系', '林文察家系']]
Checking... 國泰蔡家家系 林文明家系
[['國泰蔡家家系', '林文明家系']]
Checking... 國泰蔡家家系 林獻堂家系
0 38
found: ['國泰蔡家家系', '林文明家系', '林獻堂家系']
[['國泰蔡家家系', '林文明家系', '林獻堂家系']]
Checking... 國泰蔡家家系 林維德家系
0 38
1 82
2 668
3 5224
found: ['國泰蔡家家系', '林政雄家族', '顏雲年三子家系', '林維讓家系', '林維源家系', '林維德家系']
[['國泰蔡家家系', '林政雄家族', '顏雲年三子家系', '林維讓家系', '林維源家系', '林維德家系']]
Checking... 國泰蔡家家系 林維讓家系
0 38
1 82
found: ['國泰蔡家家系', '林政雄家族', '顏雲年三子家系', '林維讓家系']
[['國泰蔡家家系', '林政雄家族', '顏雲年三子家系', '林維讓家系']]
Checking... 國泰蔡家家系 林維源家系
0 38
1 82
2 668
found: ['國泰蔡家家系', '林政雄家族', '顏雲年三子家系', '林維讓家系', '林維源家系']
[['國泰蔡家家系', '林政雄家族', '顏雲年三子家系', '林維讓家系', '林維源家系']]
Checking... 國泰蔡家家系 顏雲年家系
0 38
1 82
found: ['國泰蔡家家系', '林政雄家族', '顏雲年三子家系', '顏雲年家系']
[['國泰蔡家家系', '林政雄家族', '顏雲年三子家系', '顏雲年家系']]
Checking... 國泰蔡家家系 顏雲年三子家系
0 38
found: ['國泰蔡家家系', '林政雄家族', '顏雲年三子家系']
[['國泰蔡家家系', '林政雄家族', '顏雲年三子家系']]
Checking... 國泰蔡家家系 顏國年家系
0 38
1 82
2 668
found: ['國泰蔡家家系', '林政雄家族', '顏雲年三子家系', '顏雲年家系',

1 176
found: ['何永家系', '楊肇嘉家系', '林月規家族', '林文察家系']
[['何永家系', '楊肇嘉家系', '林月規家族', '林文察家系']]
Checking... 何永家系 林文明家系
0 12
1 176
found: ['何永家系', '楊肇嘉家系', '林獻堂家系', '林文明家系']
[['何永家系', '楊肇嘉家系', '林獻堂家系', '林文明家系']]
Checking... 何永家系 林獻堂家系
0 12
found: ['何永家系', '楊肇嘉家系', '林獻堂家系']
[['何永家系', '楊肇嘉家系', '林獻堂家系']]
Checking... 何永家系 林維德家系
0 12
1 176
2 1144
3 6544
found: ['何永家系', '楊肇嘉家系', '林獻堂家系', '陳望曾家族', '林維源家系', '林維德家系']
[['何永家系', '楊肇嘉家系', '林獻堂家系', '陳望曾家族', '林維源家系', '林維德家系']]
Checking... 何永家系 林維讓家系
0 12
1 176
found: ['何永家系', '何義家系', '顏雲年三子家系', '林維讓家系']
[['何永家系', '何義家系', '顏雲年三子家系', '林維讓家系']]
Checking... 何永家系 林維源家系
0 12
1 176
2 1144
found: ['何永家系', '楊肇嘉家系', '林獻堂家系', '陳望曾家族', '林維源家系']
[['何永家系', '楊肇嘉家系', '林獻堂家系', '陳望曾家族', '林維源家系']]
Checking... 何永家系 顏雲年家系
0 12
1 176
found: ['何永家系', '何義家系', '顏雲年三子家系', '顏雲年家系']
[['何永家系', '何義家系', '顏雲年三子家系', '顏雲年家系']]
Checking... 何永家系 顏雲年三子家系
0 12
found: ['何永家系', '何義家系', '顏雲年三子家系']
[['何永家系', '何義家系', '顏雲年三子家系']]
Checking... 何永家系 顏國年家系
0 12
1 176
2 1144
found: ['何永家系', '楊肇嘉家系', '林獻堂家系'

In [13]:
print(len(resList), resList[0:3])

117 [['國泰蔡家家系', '林文明家系', '林文察家系'], ['國泰蔡家家系', '林文明家系'], ['國泰蔡家家系', '林文明家系', '林獻堂家系']]


In [14]:
f2fDf = pd.DataFrame(resList)
f2fDf.head()

,0,1,2,3,4,5,6
0,國泰蔡家家系,林文明家系,林文察家系,None,None,None,None
1,國泰蔡家家系,林文明家系,None,None,None,None,None
2,國泰蔡家家系,林文明家系,林獻堂家系,None,None,None,None
3,國泰蔡家家系,林政雄家族,顏雲年三子家系,林維讓家系,林維源家系,林維德家系,None
4,國泰蔡家家系,林政雄家族,顏雲年三子家系,林維讓家系,None,None,None


In [15]:
# with pd.ExcelWriter(write_file_to) as writer:
#     f2fDf.to_excel(writer, 'family2family', index=False)

#     writer.save()

# intermediate families

In [16]:
intermediateList = {}
for f2f in resList:
    useF2f = f2f.copy()
    useF2f.remove(f2f[0])
    useF2f.remove(f2f[-1])
    
    for family in useF2f:
        if family not in intermediateList:
            intermediateList[family] = 1
        else:
            intermediateList[family] += 1
            
intermediateList

{'林文明家系': 3,
 '林政雄家族': 8,
 '顏雲年三子家系': 40,
 '林維讓家系': 16,
 '林維源家系': 9,
 '顏雲年家系': 21,
 '曹永和家系': 4,
 '陳守山家系': 10,
 '辜顯榮家系': 4,
 '蔡萬才家系': 2,
 '楊肇嘉家系': 20,
 '吳鏡秋家族': 6,
 '陳啓清家系': 6,
 '李兆偉家族': 5,
 '台南高家家系': 5,
 '林獻堂家系': 15,
 '楊晏然家族': 1,
 '陳望曾家族': 6,
 '鹿港丁家': 9,
 '吳修齊家系': 2,
 '國泰蔡家家系': 6,
 '許子秋家族': 2,
 '翁家族': 2,
 '陳星恆家族': 9,
 '新竹鄭家家系': 9,
 '清水蔡惠如家系': 4,
 '嚴復家系': 4,
 '林榮三家系': 7,
 '何永家系': 3,
 '林月規家族': 2,
 '何義家系': 38,
 '台南劉家家系': 2,
 '許博允家系': 2,
 '林自西家系': 1,
 '蔡碖家族': 1,
 '顏國年家系': 1,
 '羅萬俥家族': 2,
 '楊藍洲家系': 1,
 '台南黃家家系': 1}

In [17]:
intermediateDf = pd.DataFrame(intermediateList, index=['Count'])
intermediateDf = intermediateDf.T
intermediateDf.head()

,Count
林文明家系,3
林政雄家族,8
顏雲年三子家系,40
林維讓家系,16
林維源家系,9


# family to family matrix

In [18]:
import numpy as np
matrixDf = pd.DataFrame(np.zeros((len(oldFamilies), len(newFamilies)), dtype=np.int), 
                        columns=newFamilies, index=oldFamilies)
matrixDf

,國泰蔡家家系,吳三連家族,蔡萬才家系,何義家系,何傳家系,何永家系,吳火獅家系,侯雨利家系,吳修齊家系
林文察家系,0,0,0,0,0,0,0,0,0
林文明家系,0,0,0,0,0,0,0,0,0
林獻堂家系,0,0,0,0,0,0,0,0,0
林維德家系,0,0,0,0,0,0,0,0,0
林維讓家系,0,0,0,0,0,0,0,0,0
林維源家系,0,0,0,0,0,0,0,0,0
顏雲年家系,0,0,0,0,0,0,0,0,0
顏雲年三子家系,0,0,0,0,0,0,0,0,0
顏國年家系,0,0,0,0,0,0,0,0,0
辜顯榮家系,0,0,0,0,0,0,0,0,0


In [19]:
# matrixDf.at['林文察家系', '國泰蔡家家系'] = 0
for f2f in resList:
    startF = f2f[0]
    endF = f2f[-1]
    steps = len(f2f) - 1

    matrixDf.at[endF, startF] = steps

In [20]:
matrixDf

,國泰蔡家家系,吳三連家族,蔡萬才家系,何義家系,何傳家系,何永家系,吳火獅家系,侯雨利家系,吳修齊家系
林文察家系,2,5,3,4,4,3,5,5,5
林文明家系,1,4,2,4,4,3,5,5,4
林獻堂家系,2,3,2,3,3,2,4,4,3
林維德家系,5,6,5,4,5,5,5,5,5
林維讓家系,3,4,4,2,3,3,3,3,3
林維源家系,4,5,4,3,4,4,4,4,4
顏雲年家系,3,2,3,2,3,3,3,3,1
顏雲年三子家系,2,3,3,1,2,2,2,2,2
顏國年家系,4,2,4,3,4,4,4,4,2
辜顯榮家系,3,2,4,3,4,4,4,4,3


# results prepared, write to file

In [21]:
with pd.ExcelWriter(write_file_to) as writer:
    matrixDf.to_excel(writer, 'matrix')
    intermediateDf.to_excel(writer, 'intermediate')
    f2fDf.to_excel(writer, 'family2family', index=False)

    writer.save()